In [ ]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import from_json, col, when
# from pyspark.sql.types import StructType, StringType, DoubleType

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, to_timestamp, window, expr, broadcast
from pyspark.sql.types import StructType, StringType, IntegerType, TimestampType

spark = SparkSession.builder \
    .appName("flights_stream") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.1") \
    .getOrCreate()

# Kafka source
kafka_df = (spark.readStream.format("kafka")
    .option("kafka.bootstrap.servers", "kafka:9092")
    .option("subscribe", "flights.events")
    .option("startingOffsets", "latest")
    .load())

# define schema for flights event (select columns you need)
schema = StructType() \
    .add("FL_DATE", StringType()) \
    .add("OP_CARRIER", StringType()) \
    .add("ORIGIN", StringType()) \
    .add("DEST", StringType()) \
    .add("CRS_DEP_TIME", StringType()) \
    .add("DEP_DELAY", StringType()) \
    .add("ARR_DELAY", StringType()) \
    .add("event_time", StringType()) \
    # add other fields as needed

parsed = kafka_df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")
# cast event_time properly
events = parsed.withColumn("event_time", to_timestamp("event_time"))

# read lookups (small) and broadcast
airlines = spark.read.csv("data/airlines.csv", header=True)  # or read from Cassandra
airports = spark.read.csv("data/airports.csv", header=True)

# Example: 30-minute tumbling window average arrival delay per airline
agg = (events
       .withWatermark("event_time", "1 hour")
       .groupBy(window(col("event_time"), "30 minutes"), col("OP_CARRIER"))
       .agg(
            expr("avg(cast(ARR_DELAY as double)) as avg_arr_delay"),
            expr("count(*) as num_flights")
       ))

# enrich with airline names using broadcast
agg_enriched = agg.join(broadcast(airlines), agg.OP_CARRIER == airlines.Code, "left") \
                  .select("window", "OP_CARRIER", "Name", "avg_arr_delay", "num_flights")



25/11/04 16:23:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Py4JJavaError: An error occurred while calling o45.load.
: java.lang.NoClassDefFoundError: Could not initialize class org.apache.spark.sql.kafka010.KafkaSourceProvider$
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.org$apache$spark$sql$kafka010$KafkaSourceProvider$$validateStreamOptions(KafkaSourceProvider.scala:338)
	at org.apache.spark.sql.kafka010.KafkaSourceProvider.sourceSchema(KafkaSourceProvider.scala:71)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:244)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:129)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo(DataSource.scala:129)
	at org.apache.spark.sql.execution.streaming.StreamingRelation$.apply(StreamingRelation.scala:37)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource$$anonfun$apply$1.applyOrElse(ResolveDataSource.scala:86)
	at org.apache.spark.sql.catalyst.analysis.ResolveDataSource$$anonfun$appl

In [ ]:
df_parsed = df_raw.selectExpr("CAST(value AS STRING) as json") \
    .select(from_json(col("json"), schema).alias("data")) \
    .select("data.*")

In [ ]:
df_transformed = df_parsed.withColumn(
    "delay_category",
    when(col("delay") > 0, "Delayed")
    .otherwise("On Time")
)

In [ ]:
hdfs_query = df_transformed.writeStream \
    .format("parquet") \
    .option("path", "hdfs://hadoop-namenode:9000/flights/stream_output") \
    .option("checkpointLocation", "/tmp/hdfs_checkpoint") \
    .outputMode("append") \
    .start()